In [17]:
import tensorflow as tf
import os

In [18]:
gpus = tf.config.experimental.list_physical_devices('CPU')
tf.config.experimental.set_visible_devices(gpus[0], 'CPU')

In [19]:
import cv2
import imghdr


In [20]:
import numpy as np
import matplotlib.pyplot as plt


In [16]:
train_ds

NameError: name 'train_ds' is not defined

In [4]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    'data',
    image_size=(256, 256),
    batch_size=32,
    label_mode='binary'
)

# Görüntüleri dizilere dönüştür
train_x = []
train_y = []
for images, labels in train_ds:
    for img in images:
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        train_x.append(img_array)
    for label in labels:
        train_y.append(label)

# LSTM için uygun formata dönüştür
train_x = tf.keras.preprocessing.sequence.pad_sequences(train_x, dtype='float32')
train_x = tf.reshape(train_x, (train_x.shape[0], train_x.shape[1], train_x.shape[2]*train_x.shape[3]))
train_y = tf.stack(train_y, axis=0)

# Modeli oluştur
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(128, input_shape=(train_x.shape[1], train_x.shape[2])),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Modeli derle
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Modeli eğit
hist = model.fit(train_x, train_y, batch_size=32, epochs=10, validation_split=0.2)

Found 3292 files belonging to 2 classes.
Epoch 1/10
83/83 [==============================] - 71s 788ms/step - loss: 0.6868 - accuracy: 0.5572 - val_loss: 0.6798 - val_accuracy: 0.5190
Epoch 2/10
83/83 [==============================] - 65s 781ms/step - loss: 0.6837 - accuracy: 0.5298 - val_loss: 0.6920 - val_accuracy: 0.5175
Epoch 3/10
83/83 [==============================] - 66s 793ms/step - loss: 0.6851 - accuracy: 0.5329 - val_loss: 0.6898 - val_accuracy: 0.5190
Epoch 4/10
83/83 [==============================] - 57s 688ms/step - loss: 0.6848 - accuracy: 0.5321 - val_loss: 0.6894 - val_accuracy: 0.5190
Epoch 5/10
83/83 [==============================] - 59s 716ms/step - loss: 0.6847 - accuracy: 0.5154 - val_loss: 0.6903 - val_accuracy: 0.5175
Epoch 6/10
83/83 [==============================] - 63s 758ms/step - loss: 0.6845 - accuracy: 0.5245 - val_loss: 0.6897 - val_accuracy: 0.5175
Epoch 7/10
83/83 [==============================] - 72s 864ms/step - loss: 0.6851 - accuracy: 0.5329 

In [5]:
model2 = tf.keras.Sequential([
    tf.keras.layers.LSTM(128, input_shape=(train_x.shape[1], train_x.shape[2]), return_sequences=True),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.LSTM(128),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
hist2 = model2.fit(train_x, train_y, batch_size=32, epochs=10, validation_split=0.2)

Epoch 1/10
83/83 [==============================] - 99s 1s/step - loss: 0.7909 - accuracy: 0.5097 - val_loss: 0.6920 - val_accuracy: 0.5175
Epoch 2/10
83/83 [==============================] - 102s 1s/step - loss: 0.7470 - accuracy: 0.5169 - val_loss: 0.6968 - val_accuracy: 0.4871
Epoch 3/10
83/83 [==============================] - 92s 1s/step - loss: 0.7533 - accuracy: 0.4956 - val_loss: 0.7423 - val_accuracy: 0.4871
Epoch 4/10
83/83 [==============================] - 85s 1s/step - loss: 0.7512 - accuracy: 0.4953 - val_loss: 0.7244 - val_accuracy: 0.4871
Epoch 5/10
83/83 [==============================] - 95s 1s/step - loss: 0.7357 - accuracy: 0.4922 - val_loss: 0.6903 - val_accuracy: 0.4871
Epoch 6/10
83/83 [==============================] - 103s 1s/step - loss: 0.7190 - accuracy: 0.5169 - val_loss: 0.6898 - val_accuracy: 0.5175
Epoch 7/10
53/83 [==================>...........] - ETA: 32s - loss: 0.7295 - accuracy: 0.4982

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = datagen.flow_from_directory(
        'data',
        target_size=(256, 256),
        batch_size=32,
        class_mode='binary',
        subset='training')

validation_generator = datagen.flow_from_directory(
        'data',
        target_size=(256, 256),
        batch_size=32,
        class_mode='binary',
        subset='validation')

model = keras.models.Sequential()
model.add(keras.Input(shape=(28,28))) # seq_length, input_size
model.add(layers.SimpleRNN(128, return_sequences=False, activation='relu')) # N, 128
model.add(layers.Dense(10))
print(model.summary())

# loss and optimizer
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optim = keras.optimizers.Adam(lr=0.001)
metrics = ["accuracy"]

model.compile(loss=loss, optimizer=optim, metrics=metrics)

# training
batch_size = 64
epochs = 5

H = model.fit(validation_data=(x_test, y_test), batch_size=batch_size, epochs=epochs, verbose=2)


In [40]:
data = tf.keras.utils.image_dataset_from_directory("data")

Found 3292 files belonging to 2 classes.


In [41]:
data_iterator = data.as_numpy_iterator()

In [42]:
# Get another batch of images and labels
batch = data_iterator.next()

In [43]:
batch[0].shape

(32, 256, 256, 3)

In [44]:
data = data.map(lambda x, y: (x / 255, y))

In [45]:
train_size = int(len(data) * 0.70)
val_size = int(len(data) * 0.20)
test_size = int(len(data) * 0.10)

In [46]:
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size + val_size).take(test_size)

In [51]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

model2 = keras.Sequential([
    keras.layers.TimeDistributed(keras.layers.Flatten(input_shape=(256, 256, 3))),
    keras.layers.SimpleRNN(64, return_sequences=True, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.SimpleRNN(128, return_sequences=False, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(2, activation='softmax')
])

loss = keras.losses.CategoricalCrossentropy()
optim = keras.optimizers.Adam(learning_rate=0.001)
metrics = ["accuracy"]

model2.compile(loss=loss, optimizer=optim, metrics=metrics)



In [52]:
hist = model2.fit(train, epochs=1, validation_data=val)

ValueError: in user code:

    File "c:\Users\VuralBayrakli\anaconda3\envs\env\lib\site-packages\keras\engine\training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\VuralBayrakli\anaconda3\envs\env\lib\site-packages\keras\engine\training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\VuralBayrakli\anaconda3\envs\env\lib\site-packages\keras\engine\training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\VuralBayrakli\anaconda3\envs\env\lib\site-packages\keras\engine\training.py", line 890, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "c:\Users\VuralBayrakli\anaconda3\envs\env\lib\site-packages\keras\engine\training.py", line 948, in compute_loss
        return self.compiled_loss(
    File "c:\Users\VuralBayrakli\anaconda3\envs\env\lib\site-packages\keras\engine\compile_u

In [77]:
model2 = keras.Sequential([
    keras.layers.TimeDistributed(keras.layers.SimpleRNN(256, activation='relu'),
                                  input_shape=(256, 256, 3)),
    keras.layers.TimeDistributed(keras.layers.Flatten()),
    keras.layers.SimpleRNN(256, return_sequences=False, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])
model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

hist = model2.fit(train, epochs=10, validation_data=val)

Epoch 1/10
 7/72 [=>............................] - ETA: 22:46 - loss: 0.6918 - accuracy: 0.6205

KeyboardInterrupt: 

In [49]:
epochs = 10
history = model.fit(train, validation_data=val, epochs=epochs)


Epoch 1/10


InvalidArgumentError: Graph execution error:

Input to reshape is a tensor with 6291456 values, but the requested shape requires a multiple of 784
	 [[{{node Reshape}}]]
	 [[IteratorGetNext]] [Op:__inference_train_function_3038]

In [27]:
scaled_iterator =  data.as_numpy_iterator()

In [28]:
batch = scaled_iterator.next()

In [39]:
model1 = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(256, 256, 3)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.BatchNormalization())
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(train, epochs=10, batch_size=32, validation_data=val)


Epoch 1/10


ValueError: in user code:

    File "c:\Users\VuralBayrakli\anaconda3\envs\env\lib\site-packages\keras\engine\training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\VuralBayrakli\anaconda3\envs\env\lib\site-packages\keras\engine\training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\VuralBayrakli\anaconda3\envs\env\lib\site-packages\keras\engine\training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\VuralBayrakli\anaconda3\envs\env\lib\site-packages\keras\engine\training.py", line 889, in train_step
        y_pred = self(x, training=True)
    File "c:\Users\VuralBayrakli\anaconda3\envs\env\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\VuralBayrakli\anaconda3\envs\env\lib\site-packages\keras\engine\input_

In [ ]:
model = keras.models.Sequential()
model.add(keras.Input(shape=(28,28))) # seq_length, input_size
model.add(layers.SimpleRNN(128, return_sequences=False, activation='relu')) # N, 128
model.add(layers.Dense(10))
print(model.summary())

# loss and optimizer
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optim = keras.optimizers.Adam(lr=0.001)
metrics = ["accuracy"]

model.compile(loss=loss, optimizer=optim, metrics=metrics)

# training
batch_size = 64
epochs = 5

H = model.fit(validation_data=(x_test, y_test), , epochs=epochs, verbose=2)

In [13]:
logdir = "logs"

In [14]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

In [ ]:
hist = model.fit(train, validation_data=val, epochs=3, callbacks=[tensorboard_callback])

In [19]:
data??

Type:           BatchDataset
String form:    <BatchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>
Length:         103
File:           c:\users\vuralbayrakli\anaconda3\envs\env\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py
Source:        
class BatchDataset(UnaryDataset):
  """A `Dataset` that batches contiguous elements from its input."""

  def __init__(self, input_dataset, batch_size, drop_remainder, name=None):
    """See `Dataset.batch()` for details."""
    self._input_dataset = input_dataset
    self._batch_size = ops.convert_to_tensor(
        batch_size, dtype=dtypes.int64, name="batch_size")
    self._drop_remainder = ops.convert_to_tensor(
        drop_remainder, dtype=dtypes.bool, name="drop_remainder")

    constant_drop_remainder = tensor_util.constant_value(self._drop_remainder)
    # pylint: disable=protected-access
    if constant_drop_remainder:
      # NOTE(